In [2]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np

In [23]:
girder = gc.GirderClient(host="f9e93620.ngrok.io")

# Plan

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [x] userA creates an applet with an activity-set-url (url TBD)
- [ ] get userA's applets and make sure we have the applet they just created
- [ ] create userB
- [ ] userA invites userB to the applet
- [ ] make sure userB has an invite
- [ ] userB accepts their invite
- [ ] list userB's applets
- [ ] userA invites userC (by email) to the applet
- [ ] create userC with the same email
- [ ] check that userC has an invite
- [ ] accept userC's applet invite
- [ ] check userC's applets
- [ ] userA, B and C submit responses to each activity in the applet
- [ ] userA can see user's B & C's data
- [ ] get responses for each user in the last 7 days
- [ ] user B removes the applet without removing data
- [ ] user C removes the applet AND removes data
- [x] delete userB and userC
- [ ] userA deactivites their applet
- [ ] delete userA


# Create a new user

In [24]:
def testCreateUser():
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [25]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')

{'_accessLevel': 2,
 '_id': '5d82902c4f5fa58f75e86880',
 '_modelType': 'user',
 'admin': False,
 'created': '2019-09-18T20:14:35.713000+00:00',
 'creatorId': '5d82902c4f5fa58f75e86880',
 'email': 'testemail385918@testemail.com',
 'emailVerified': False,
 'firstName': 'test',
 'lastName': 'user',
 'login': 'testuser385918',
 'otp': False,
 'public': False,
 'size': 0,
 'status': 'enabled'}

# Get applets when the user doesn't have any

In [26]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [27]:
getAppletsNewUser(new_user)

1

# Add an applet

In [28]:
def addApplet(new_user):
    new_user = testCreateUser()
    girder.authenticate(username=new_user['login'], password='password')

    # TODO: create an activity-set that JUST for testing.
    # make sure it has all the weird qualities that can break


    # for now, lets do the mindlogger demo
    activitySetUrl = 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'
    randomAS = np.random.randint(1000000)
    ar = girder.post('applet', parameters=dict(activitySetUrl = activitySetUrl, name='testActivitySet{}'.format(randomAS)))

    assert ar['_id'], 'there is no ID!'
    assert ar['meta']['activitySet']['url'] == activitySetUrl, 'the URLS do not match! {} {}'.format(ar['meta']['activitySet']['url'], activitySetUrl)
    return ar


In [29]:
ar = addApplet(new_user)
ar

{'_id': '5d82904d4f5fa58f75e86883',
 'access': {'groups': [],
  'users': [{'flags': [], 'id': '5d82904c4f5fa58f75e86881', 'level': 2}]},
 'baseParentId': '5c8fc74096eff13180efa7b7',
 'baseParentType': 'collection',
 'created': '2019-09-18T20:15:09.908611+00:00',
 'creatorId': '5d82904c4f5fa58f75e86881',
 'description': '',
 'lowerName': 'testactivityset80710',
 'meta': {'activitySet': {'_id': 'activitySet/5d82904d4f5fa58f75e86882',
   'url': 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'},
  'applet': {}},
 'name': 'testActivitySet80710',
 'parentCollection': 'collection',
 'parentId': '5c8fc74096eff13180efa7b7',
 'public': True,
 'roles': {'editor': {'groups': [{'id': '5d82904d4f5fa58f75e86885'}],
   'users': []},
  'manager': {'groups': [{'id': '5d82904d4f5fa58f75e86886'}], 'users': []},
  'reviewer': {'groups': [{'id': '5d82904e4f5fa58f75e86887', 'subject': None}],
   'users': []},
  'user': {'groups': [{'id': '5d82904d4f5fa58f75e86

# get Applet by ID

want to check recently added applet here

In [15]:
new_user = testCreateUser()
girder.authenticate(username=new_user['login'], password='password')
girder.get('applet/{}'.format(ar['_id']))

{'activities': {'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/MindLoggerDemo/MindLoggerDemo_schema': {'@context': ['https://raw.githubusercontent.com/ReproNim/schema-standardization/master/contexts/generic',
    'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/MindLoggerDemo/MindLoggerDemo_context'],
   '@id': '/MindLoggerDemo_schema',
   '@type': ['https://raw.githubusercontent.com/ReproNim/schema-standardization/master/schemas/Activity'],
   '_id': 'activity/5d6ee74f8f66701eb8166017',
   'http://schema.org/description': [{'@language': 'en',
     '@value': 'This activity contains one of each type of widget supported by the MindLogger mobile app.'}],
   'http://schema.org/schemaVersion': [{'@language': 'en', '@value': '0.0.1'}],
   'http://schema.org/url': [{'@language': 'en',
     '@value': 'https://raw.githubusercontent.com/ReproNim/schema-standardization/master/activities/MindLoggerDemo/MindLoggerDemo_schema'

# Delete a user

In [11]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [12]:
testDeleteUser(new_user)

1